In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
import LoadDataset as LD
import numpy as np
import scipy.misc
from scipy.misc import imresize
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

In [12]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 6
print(epochs)

6


In [16]:
n_categories = 2
categories = np.array(['c0', 'c1'])

load_data = LD.LoadData('/Users/mdabdulkadir/WS1819/ADAI/imagetest') # give the directory address.
X_dat ,Y_dat = load_data.generate_data_2cls()

X_dat, Y_dat = load_data.shuffle_data(X_dat,Y_dat)

Dataset directory added
['c0', 'c1', 'c2']


AttributeError: 'LoadData' object has no attribute 'generate_data_2cls'

In [4]:
X = np.array(X_dat)
Y = np.array(Y_dat)
X.shape

(22424, 227, 227, 3)

In [18]:
Y[0:50]

array([7, 6, 0, 6, 4, 8, 3, 7, 1, 9, 8, 1, 8, 1, 5, 5, 8, 2, 8, 2, 8, 7,
       2, 1, 2, 4, 6, 5, 4, 4, 3, 5, 7, 9, 3, 0, 1, 5, 5, 4, 8, 6, 7, 7,
       9, 5, 5, 0, 2, 5])

In [19]:
def pca(img, alpha_std=0.1):
    orig_img = img.astype(float).copy()
    img = img / 255.0  # rescale to 0 to 1 range
    # flatten image to columns of RGB
    img_rs = img.reshape(-1, 3)
    # img_rs shape (640000, 3)
    img_centered = img_rs - np.mean(img_rs, axis=0)
    # paper says 3x3 covariance matrix
    img_cov = np.cov(img_centered, rowvar=False)
    # eigen values and eigen vectors
    eig_vals, eig_vecs = np.linalg.eigh(img_cov)
    # sort values and vector
    sort_perm = eig_vals[::-1].argsort()
    eig_vals[::-1].sort()
    eig_vecs = eig_vecs[:, sort_perm]

    # get [p1, p2, p3]
    m1 = np.column_stack((eig_vecs))

    # get 3x1 matrix of eigen values multiplied by random variable draw from normal
    # distribution with mean of 0 and standard deviation of 0.1
    m2 = np.zeros((3, 1))
    # according to the paper alpha should only be draw once per augmentation (not once per channel)
    alpha = np.random.normal(0, alpha_std)

    # broad cast to speed things up
    m2[:, 0] = alpha * eig_vals[:]

    # this is the vector that we're going to add to each pixel in a moment
    add_vect = np.matrix(m1) * np.matrix(m2)

    for idx in range(3):   # RGB
        orig_img[..., idx] += add_vect[idx]
    orig_img = np.clip(orig_img, 0.0, 255.0)
    orig_img = orig_img.astype(np.uint8)
    return orig_img


In [9]:
(x_train, y_train) = (X[0:20000], Y[0:20000])
(x_test, y_test) = (X[20000:], Y[20000:])
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(img_height, img_width, channel)

x_train = pca(x_train)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(x_train.shape)

227 227 3
(20000, 227, 227, 3)


In [13]:
y_train[1]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [10]:
# # for cifar10 dataset
# # Load CIFAR10 Data
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# print(x_train.shape)

# img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# # convert to one hot encoing
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)
# print(y_train.shape)

(50000, 32, 32, 3)
(50000, 10)


In [10]:
# AlexNet Define the Model
model = Sequential()
# model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', padding='valid', input_shape=(img_height, img_width, channel,)))
# for original Alexnet
model.add(Conv2D(96, (3,3), strides=(2,2), activation='relu', padding='same', input_shape=(img_height, img_width, channel,)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Conv2D(256, (5,5), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Conv2D(384, (3,3), activation='relu', padding='same'))
model.add(Conv2D(384, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [11]:
# print the model summary
model.summary()

# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Train the Model

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 114, 114, 96)      2688      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 57, 57, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 57, 57, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 57, 57, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 256)       1024      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 384)       885120    
__________

In [12]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 14.482347881833318
Test accuracy: 0.10148514852714617
